# Notebook for model development
This notebook should be used to develop the model submodules locally. Note, that model relevant changes have to be made in the model subclasses. Additional code in this document is only used to get a better understanding of the dataflow and debug some outputs.

## 1: Set up environment
### Install dependencies
This will use the automation `install_dependencies.sh` in order to install necessary packages. Additional packages for development can be installed afterwards.

In [ ]:
! cd .. && bash automation/train_model/install_dependencies.sh

### Import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import gc

## 2: Get the data ready
In this step, the data will be imported and preprocessed.

### Download Dataset
The dataset has to be imported using the submodule `model/import_data.py`.

In [2]:
from model.import_dataset import import_dataset

dataset_id = "ea93eb20-616c-4ad2-9d82-cca701766612"
images, labels, uids = import_dataset(dataset_id)

del dataset_id
gc.collect()

### Inspect dataset

In [ ]:
print("Number of images: ", images.shape[0])
print("Labels from ", np.min(labels), " to ", np.max(labels), "\n")

def plot_image(image, label):
  plt.imshow(image)
  plt.title(f'distance={label}')
  plt.show()

# Plot some images
num_images_to_plot = 1
for i in range(0, num_images_to_plot):
  plot_image(images[i], labels[i])

# Plot histogram of labels
plt.hist(labels, edgecolor='black')  # Erstelle das Histogramm mit 10 Bins
plt.title('Distribution of labels in dataset')
plt.xlabel('Distance [mm]')
plt.ylabel('n')
plt.show()

del num_images_to_plot, i
gc.collect()

### Preprocess dataset
The dataset is preprocessed using the submodule `model/preprocess_data.py`.

In [4]:
from model.preprocess_data import preprocess_data

test_split = 0.2
random_state = 0
train_x, train_y, train_uids, test_x, test_y, test_uids = \
    preprocess_data(images, labels, uids, test_split, random_state)

del images, labels, uids
gc.collect()

In [ ]:
print("\nTrainingsdaten: ")
mean = np.mean(train_x)
std = np.std(train_x)
max = np.max(train_x)
min = np.min(train_x)
print(f"Anzahl: {train_x.shape[0]}")
print(f"Durchschnittlicher Pixelwert: {mean:.2f}")
print(f"Standardabweichung der Pixelwerte: {std:.2f}")
print(f"Höchster Pixelwert: {max:.2f}")
print(f"Niedrigster Pixelwert: {min:.2f}")
print(f"Bildgröße (Höhe x Breite): {train_x.shape[1]} x {train_x.shape[2]}")
print(f"Anzahl der Klassen: {len(np.unique(train_y))}")

plot_image(train_x[0], train_y[0])

print("\nTestdaten: ")
mean = np.mean(test_x)
std = np.std(test_x)
max = np.max(test_x)
min = np.min(test_x)
print(f"Anzahl: {test_x.shape[0]}")
print(f"Durchschnittlicher Pixelwert: {mean:.2f}")
print(f"Standardabweichung der Pixelwerte: {std:.2f}")
print(f"Höchster Pixelwert: {max:.2f}")
print(f"Niedrigster Pixelwert: {min:.2f}")
print(f"Bildgröße (Höhe x Breite): {test_x.shape[1]} x {test_x.shape[2]}")
print(f"Anzahl der Klassen: {len(np.unique(test_y))}")

plot_image(test_x[0], test_y[0])

# Plot histogram of labels
print("\nLabel distribution in training and test set:")
plt.hist([train_y, test_y], label=['Train', 'Test'],
          edgecolor='black', color=['blue', 'orange'])
plt.title('Distribution of labels in dataset')
plt.xlabel('Distance [mm]')
plt.ylabel('n')
plt.legend()
plt.show()

del mean, min, max, std
gc.collect()

## 3: Model

### Create model
The model is created using `model/create_model.py`

In [ ]:
from model.create_model import create_model

model = create_model(dropout=0.2)

### Inspect model

In [ ]:
model.summary()

### Model training
The model is trained using `model_fit_model.py`
This should ideally be done using the workflow.

In [ ]:
from model.fit_model import fit_model

history = fit_model(model, x_train, y_train,
              optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'],
              epochs=5)

In [ ]:
print(f"Final loss: {history.history['loss'][-1]:.4f}")
print(f"Final accuracy: {history.history['accuracy'][-1]:.4f}")

# Trainings- und Validierungsverlust
plt.figure(1)
plt.plot(history.history['loss'], label='Training Loss')
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss per epoch')
plt.xlabel('Epopch')
plt.ylabel('Loss')
plt.legend()

# Genauigkeit (Accuracy) plotten
plt.figure(2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
if 'val_accuracy' in history.history:
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

### Evaluate model
The model is evaluated using `model/evaluate_model.py`

In [ ]:
from model.evaluate_model import evaluate_model

test_loss, test_acc = evaluate_model(model, x_test, y_test)

In [ ]:
print(f'Test accuracy: {test_acc:.4f}')
print(f'Test accuracy: {test_acc:.4f}')


### Save model

In [ ]:
model.save('model.keras')